In [1]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding, Masking, Dropout, InputLayer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [2]:
dataset = pd.read_csv("../data/archive/TranslateTest.csv", sep=";", encoding="utf-8")

FileNotFoundError: [Errno 2] No such file or directory: '../data/archive/TranslateTest.csv'

In [3]:
dataset = dataset.fillna(" ")

In [5]:
dataset['Симптомы'] = dataset[["Симптом 1","Симптом 2","Симптом 3","Симптом 4","Симптом 5","Симптом 6","Симптом 7","Симптом 8","Симптом 9",
                               "Симптом 10","Симптом 11","Симптом 12","Симптом 13","Симптом 14","Симптом 15","Симптом 16","Симптом 17"]].agg(' '.join, axis=1)

# Удаляем лишние колонки
dataset = dataset[['Болезнь', 'Симптомы']]

In [6]:
dataset.to_csv("../data/archive/example.csv")
dataset

,Болезнь,Симптомы
0,Грибковая инфекция,зуд кожная сыпь узловые кожные высыпания ди...
1,Грибковая инфекция,кожная сыпь узловые кожные высыпания дисхро...
2,Грибковая инфекция,зуд узловые кожные высыпания дисхромные пятн...
3,Грибковая инфекция,зуд кожная сыпь дисхромные пятна ...
4,Грибковая инфекция,зуд кожная сыпь узловые кожные высыпания ...
...,...,...
299,Импетиго,высокая температура волдыри красная язвы во...
300,Импетиго,кожная сыпь волдыри красная язвы вокруг нос...
301,Импетиго,кожная сыпь высокая температура красная язв...
302,Импетиго,кожная сыпь высокая температура волдыри же...


In [7]:
# Токинезация симптомов, то есть представление их в виде множества чисел
max_words_count=500
tokenizer=Tokenizer(num_words=max_words_count, split=" ")
tokenizer.fit_on_texts(dataset['Симптомы'])

In [8]:
# "Подгоняем" размер множеств чисел под одну длину 
data_x = tokenizer.texts_to_sequences(dataset['Симптомы'])

In [9]:
data_x

[[27, 25, 26, 202, 203, 91, 204, 37],
 [25, 26, 202, 203, 91, 204, 37],
 [27, 202, 203, 91, 204, 37],
 [27, 25, 26, 204, 37],
 [27, 25, 26, 202, 203, 91],
 [81, 82, 205, 17, 206, 207, 12],
 [205, 17, 206, 207, 12],
 [81, 82, 17, 206, 207, 12],
 [81, 82, 205, 206, 207, 12],
 [81, 82, 205, 17],
 [1, 2, 83, 63, 84, 48, 159, 5, 30, 1, 2, 20],
 [1, 2, 83, 84, 48, 159, 5, 30, 1, 2, 20],
 [1, 2, 83, 63, 5, 30, 1, 2, 20],
 [1, 2, 83, 63, 84, 48, 159, 30, 1, 2, 20],
 [1, 2, 83, 63, 84, 48, 159, 5, 1, 2, 20],
 [1, 2, 83, 63, 84, 48, 159, 5, 30],
 [63, 84, 48, 159, 5, 30, 1, 2, 20],
 [27, 5, 13, 14, 10, 3, 8, 1, 2, 9, 15, 12],
 [5, 13, 14, 10, 3, 8, 1, 2, 9, 15, 12],
 [27, 13, 14, 10, 3, 8, 1, 2, 9, 15, 12],
 [27, 5, 10, 3, 8, 1, 2, 9, 15, 12],
 [27, 5, 13, 14, 3, 8, 1, 2, 9, 15, 12],
 [27, 5, 13, 14, 10, 1, 2, 9, 15, 12],
 [27, 5, 13, 14, 10, 3, 8, 15, 12],
 [27, 5, 13, 14, 10, 3, 8, 1, 2, 9],
 [27, 25, 26, 1, 2, 83, 94, 70, 71, 181, 182, 70, 71],
 [27, 1, 2, 83, 94, 70, 71, 181, 182, 70, 71],
 

In [10]:
max_len = max([len(x) for x in data_x])
print(max_len)
data_x = pad_sequences(data_x, maxlen=max_len)

30


In [11]:
data_x

array([[  0,   0,   0, ...,  91, 204,  37],
       [  0,   0,   0, ...,  91, 204,  37],
       [  0,   0,   0, ...,  91, 204,  37],
       ...,
       [  0,   0,   0, ..., 199, 200, 201],
       [  0,   0,   0, ..., 199, 200, 201],
       [  0,   0,   0, ...,  84, 197, 198]], dtype=int32)

In [12]:
# Кодирование меток заболеваний
label_encoder = LabelEncoder()
data_y = label_encoder.fit_transform(dataset['Болезнь'])

In [13]:
data_y

array([19, 19, 19, 19, 19,  4,  4,  4,  4,  4, 10, 10, 10, 10, 10, 10, 10,
       37, 37, 37, 37, 37, 37, 37, 37, 34, 34, 34, 34, 34, 34, 40, 40, 40,
       40, 40, 40, 40, 35, 35, 35, 35, 35, 21, 21, 21, 21, 21, 21, 21, 21,
       21, 11, 11, 11, 11, 11,  6,  6,  6,  6,  6,  6,  6,  0,  0,  0,  0,
        0,  0, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 38, 38, 38, 38, 38,
       38, 30, 30, 30, 30, 30, 23, 23, 23, 23, 23, 23, 23, 23, 23, 26, 26,
       26, 26, 26, 26, 26, 26,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9, 20,
       20, 20, 20, 20, 20, 20, 20, 20, 20,  7,  7,  7,  7,  7,  7,  7,  7,
        7, 39, 39, 39, 39, 39, 39, 39, 39, 39, 12, 12, 12, 12, 12, 12, 12,
       12, 12, 15, 15, 15, 15, 15, 15, 15, 13, 13, 13, 13, 13, 13, 13, 13,
       13, 13, 14, 14, 14, 14, 14, 14, 14, 14, 14,  3,  3,  3,  3,  3,  3,
        3,  3, 36, 36, 36, 36, 36, 36, 36, 36, 36, 32, 32, 32, 32, 32, 32,
       32, 32, 32, 31, 31, 31, 31, 31, 31, 31, 31, 31, 22, 22, 22, 22, 22,
       22, 29, 29, 29, 29

In [14]:
# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(data_x, data_y, test_size=0.2, random_state=42)

In [15]:
# Создание модели LSTM
model = Sequential()
model.add(InputLayer(shape=(max_len,)))
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=128))
model.add(LSTM(64))
model.add(Dropout(0.5))
model.add(Dense(len(label_encoder.classes_), activation='softmax'))

# Компиляция модели
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Обучение модели
model.fit(X_train, y_train, epochs=9, batch_size=2, validation_data=(X_test, y_test))

Epoch 1/9
122/122 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 0.0959 - loss: 3.6845 - val_accuracy: 0.4426 - val_loss: 3.5012
Epoch 2/9
122/122 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.3356 - loss: 3.1796 - val_accuracy: 0.5410 - val_loss: 2.3957
Epoch 3/9
122/122 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.6464 - loss: 2.0498 - val_accuracy: 0.7869 - val_loss: 1.4674
Epoch 4/9
122/122 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.8568 - loss: 1.1501 - val_accuracy: 0.8525 - val_loss: 0.9929
Epoch 5/9
122/122 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.8550 - loss: 0.8754 - val_accuracy: 0.8852 - val_loss: 0.6484
Epoch 6/9
122/122 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.9307 - loss: 0.5888 - val_accuracy: 0.9180 - val_loss: 0.4972
Epoch 7/9
122/122 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.9859 - loss: 0.3258 - val_accuracy: 0.9508 - val_loss: 0.3784
Epoch 8/9
122/122 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.9754 - loss: 0.2801 - val_accuracy: 0.9836 - 

In [46]:
# Тестовая классификация
# На сайте будет подобное иcпользование, где вместо test_symp будет подоваться множество симптомов юзера
test_symp = 'постоянное чихание озноб мокрота головная боль давление в пазухах насморк скопление'
t = tokenizer.texts_to_sequences([test_symp])
t_pad = pad_sequences(t, maxlen = max_len)

In [47]:
#Получение результатов классификации 
res = model.predict(t_pad)
label_res = {
    "Болезнь": label_encoder.classes_,
    "values": res[0]
}
res_frame = pd.DataFrame(label_res)
res_frame.sort_values('values', ascending=False)
#Таблица с результатами классификации в порядке убывания вероятности

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step


,Болезнь,values
32,Простуда,0.159684
0,Гипертония,0.137204
10,ГЭРБ,0.126084
7,Брюшной тиф,0.074882
26,Малярия,0.068008
38,Шейный спондилез,0.066260
16,Гипертиреоз,0.060934
11,Гастроэнтерит,0.051265
3,Алкогольный гепатит,0.029964
30,Паралич (кровоизлияние в мозг),0.024580


In [17]:
#model.save('path/to/model.h5') # сохранение модели 
#new_model = tf.keras.models.load_model('path/to/model.h5') # загрузка модели 